# CRF
In this lab session, you are going to train and test a linear-chain CRF model. Before starting, specify "python 2" in the environment parameters.

In [25]:
!pip install python-crfsuite

In [26]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.22.2.post1


# Let's use CoNLL 2002 data to build a NER system

CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [27]:
nltk.download('conll2002')
nltk.corpus.conll2002.fileids()

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [28]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

CPU times: user 2.2 s, sys: 87 ms, total: 2.29 s
Wall time: 2.29 s


Data format:

In [29]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## Features

Next, define some features. In this example we use word identity, word suffix, word shape and word POS tag; also, some information from nearby words is used. 

This makes a simple baseline, but you certainly can add and remove some features to get (much?) better results - experiment with it.

In [30]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

This is what word2features extracts:

In [31]:
sent2features(train_sents[0])[0]

['bias',
 'word.lower=melbourne',
 'word[-3:]=rne',
 'word[-2:]=ne',
 'word.isupper=False',
 'word.istitle=True',
 'word.isdigit=False',
 'postag=NP',
 'postag[:2]=NP',
 'BOS',
 '+1:word.lower=(',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=Fpa',
 '+1:postag[:2]=Fp']

Extract the features from the data:

In [32]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 3.07 s, sys: 218 ms, total: 3.29 s
Wall time: 3.3 s


## Train the model

To train the model, we create pycrfsuite.Trainer, load the training data and call 'train' method. 
First, create pycrfsuite.Trainer and load the training data to CRFsuite:

In [33]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 2.92 s, sys: 17.6 ms, total: 2.94 s
Wall time: 2.95 s


Set training parameters. We will use L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [34]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

Possible parameters for the default training algorithm:

In [35]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

Train the model:

In [36]:
%%time
trainer.train('conll2002-esp.crfsuite')

CPU times: user 22.8 s, sys: 23.7 ms, total: 22.8 s
Wall time: 22.8 s


trainer.train saves model to a file:

In [37]:
!ls -lh ./conll2002-esp.crfsuite

-rw-r--r-- 1 root root 601K Mar 16 10:39 ./conll2002-esp.crfsuite


We can also get information about the final state of the model by looking at the trainer's logparser. If we had tagged our input data using the optional group argument in add, and had used the optional holdout argument during train, there would be information about the trainer's performance on the holdout set as well. 

In [38]:
trainer.logparser.last_iteration

{'active_features': 11346,
 'error_norm': 1262.912078,
 'feature_norm': 79.110017,
 'linesearch_step': 1.0,
 'linesearch_trials': 1,
 'loss': 14807.577946,
 'num': 50,
 'scores': {},
 'time': 0.423}

We can also get this information for every step using trainer.logparser.iterations

In [39]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'num': 50, 'scores': {}, 'loss': 14807.577946, 'feature_norm': 79.110017, 'error_norm': 1262.912078, 'active_features': 11346, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.423}


## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [40]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

Let's tag a sentence to see how it works:

In [41]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

La Coruña , 23 may ( EFECOM ) .
Predicted: B-LOC I-LOC O O O O B-ORG O O
Correct:   B-LOC I-LOC O O O O B-ORG O O


## Evaluate the model

In [42]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

Predict entity labels for all sentences in our testing set ('testb' Spanish data):

In [43]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 523 ms, sys: 18 ms, total: 541 ms
Wall time: 558 ms


..and check the result. Note this report is not comparable to results in CONLL2002 papers because here we check per-token results (not per-entity). Per-entity numbers will be worse.  

In [44]:
print(bio_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-LOC       0.78      0.75      0.76      1084
       I-LOC       0.66      0.60      0.63       325
      B-MISC       0.69      0.47      0.56       339
      I-MISC       0.61      0.49      0.54       557
       B-ORG       0.79      0.81      0.80      1400
       I-ORG       0.80      0.79      0.80      1104
       B-PER       0.82      0.87      0.84       735
       I-PER       0.87      0.93      0.90       634

   micro avg       0.78      0.76      0.77      6178
   macro avg       0.75      0.71      0.73      6178
weighted avg       0.77      0.76      0.76      6178
 samples avg       0.09      0.09      0.09      6178



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Let's check what classifier learned

In [45]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B-ORG  -> I-ORG   8.631963
I-ORG  -> I-ORG   7.833706
B-PER  -> I-PER   6.998706
B-LOC  -> I-LOC   6.913675
I-MISC -> I-MISC  6.129735
B-MISC -> I-MISC  5.538291
I-LOC  -> I-LOC   4.983567
I-PER  -> I-PER   3.748358
B-ORG  -> B-LOC   1.727090
B-PER  -> B-LOC   1.388267
B-LOC  -> B-LOC   1.240278
O      -> O       1.197929
O      -> B-ORG   1.097062
I-PER  -> B-LOC   1.083332
O      -> B-MISC  1.046113

Top unlikely transitions:
I-PER  -> B-ORG   -2.056130
I-LOC  -> I-ORG   -2.143940
B-ORG  -> I-MISC  -2.167501
I-PER  -> I-ORG   -2.369380
B-ORG  -> I-PER   -2.378110
I-MISC -> I-PER   -2.458788
B-LOC  -> I-PER   -2.516414
I-ORG  -> I-MISC  -2.571973
I-LOC  -> B-PER   -2.697791
I-LOC  -> I-PER   -3.065950
I-ORG  -> I-PER   -3.364434
O      -> I-PER   -7.322841
O      -> I-MISC  -7.648246
O      -> I-ORG   -8.024126
O      -> I-LOC   -8.333815


We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [46]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
8.886516 B-ORG  word.lower=efe-cantabria
8.743642 B-ORG  word.lower=psoe-progresistas
5.769032 B-LOC  -1:word.lower=cantabria
5.195429 I-LOC  -1:word.lower=calle
5.116821 O      word.lower=mayo
4.990871 O      -1:word.lower=día
4.910915 I-ORG  -1:word.lower=l
4.721572 B-MISC word.lower=diversia
4.676259 B-ORG  word.lower=telefónica
4.334354 B-ORG  word[-2:]=-e
4.149862 B-ORG  word.lower=amena
4.141370 B-ORG  word.lower=terra
3.942852 O      word.istitle=False
3.926397 B-ORG  word.lower=continente
3.924672 B-ORG  word.lower=acesa
3.888706 O      word.lower=euro
3.856445 B-PER  -1:word.lower=según
3.812373 B-MISC word.lower=exteriores
3.807582 I-MISC -1:word.lower=1.9
3.807098 B-MISC word.lower=sanidad

Top negative:
-1.965379 O      word.lower=fundación
-1.981541 O      -1:word.lower=británica
-2.118347 O      word.lower=061
-2.190653 B-PER  word[-3:]=nes
-2.226373 B-ORG  postag=SP
-2.226373 B-ORG  postag[:2]=SP
-2.260972 O      word[-3:]=uia
-2.384920 O      -1:word.lower

Some observations:

* **8.743642 B-ORG  word.lower=psoe-progresistas** - the model remembered names of some entities - maybe it is overfit, or maybe our features are not adequate, or maybe remembering is indeed helpful;
* **5.195429 I-LOC  -1:word.lower=calle**: "calle" is a street in Spanish; model learns that if a previous word was "calle" then the token is likely a part of location;
* **-3.529449 O      word.isupper=True**, ** -2.913103 O      word.istitle=True **: UPPERCASED or TitleCased words are likely entities of some kind;
* **-2.585756 O      postag=NP** - proper nouns (NP is a proper noun in the Spanish tagset) are often entities.

# Coding your own CRF inference routine

To help you, we use a library, named flexcrf, that provide some inference routines you will use to test your viterbi algorithm. With the following command, you donwload and unzip it.

In [47]:
!if [[ ! -d flexcrf_tp ]]; then wget http://stelat.eu/wp-content/uploads/2020/03/flexcrf_tp.zip && unzip flexcrf_tp.zip;fi

In [50]:
pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


Here, we import some functions


In [51]:
import cPickle as pickle

import numpy as np
from pycrfsuite import Tagger
from flexcrf_tp.models.linear_chain import (_feat_fun_values,
                                            _compute_all_potentials,
                                            _forward_score,
                                            _backward_score,
                                            _partition_fun_value,
                                            _posterior_score)

from flexcrf_tp.crfsuite2flexcrf import convert_data_to_flexcrf

SyntaxError: ignored

#Viterbi decoder
Now you can complete the viterbi_decoder function:

In [ ]:


# -- Define vitrebi_decoder here:

def viterbi_decoder(m_xy, n=None, log_version=True):
    """
    Performs MAP inference, determining $y = \argmax_y P(y|x)$, using the
    Viterbi algorithm.

    Parameters
    ----------
    m_xy : ndarray, shape (n_obs, n_labels, n_labels)
        Values of log-potentials ($\log M_i(y_{i-1}, y_i, x)$)
        computed based on feature functions f_xy and/or user-defined potentials
        `psi_xy`. At t=0, m_xy[0, 0, :] contains values of $\log M_1(y_0, y_1)$
        with $y_0$ the fixed initial state.

    n : integer, default=None
        Time position up to which to decode the optimal sequence; if not
        specified (default) the score is computed for the whole sequence.

    Returns
    -------
    y_pred : ndarray, shape (n_obs,)
        Predicted optimal sequence of labels.

    TODO: Cythonise this function for more efficiency.
    """

    if n is None:
        n = m_xy.shape[0]

    # Here we provide the temporary variables required by the viterbi algorithm.
    n_labels = m_xy.shape[2]
    y_pred = np.empty(n, dtype=int)
    delta = np.empty((n, n_labels))
    delta[0, :] = m_xy[0, 0, :]
    btrack = np.empty((n, n_labels), dtype=int)

    # Viterbi scores
    for m in xrange(1, n) : 
      score = delta[m-1, :] + m_xy[m, :, :].T
      btrack[m, :] = np.argmax(score, axis=1)
      delta[m,:] = score[np.r_[:n_labels], btrack[m, :]]
    #YOUR CODE HERE
    
    # Backtracking
    #YOUR CODE HERE
    y_pred[n-1] = np.argmax(delta[n-1, :])
    for m in range(n-2, -1,-1):
      y_pred[m] = btrack[m+1, y_pred[m+1]]
    
    return y_pred

# Test your Viterbi decoder
Check if you viterbi decoder provide the same output as pycrfsuite.

In [ ]:


# -- Load data and crfsuite model and convert them-------------------------

RECREATE = True  # set to True to recreate flexcrf data with new model

CRFSUITE_MODEL_FILE = './conll2002-esp.crfsuite'
CRFSUITE_MODEL_INFO_FILE = './conll2002-esp.crfsuite-model-info.dump'

CRFSUITE_TEST_DATA_FILE = './conll2002-esp_crfsuite-test-data.dump'
FLEXCRF_TEST_DATA_FILE = './conll2002-esp_flexcrf-test-data.dump'

# crfsuite model
tagger = Tagger()
tagger.open(CRFSUITE_MODEL_FILE)
model = tagger.info()
#model = pickle.load(open(CRFSUITE_MODEL_INFO_FILE))
#print "model loaded."
data={'X': X_test, 'y': y_test}
#data = pickle.load(open(CRFSUITE_TEST_DATA_FILE))
#print "test data loaded."

if RECREATE:
    dataset, thetas = convert_data_to_flexcrf(data, model, n_seq=3)
    pickle.dump({'dataset': dataset, 'thetas': thetas},
                open(FLEXCRF_TEST_DATA_FILE, 'wb'))
else:
    dd = pickle.load(open(FLEXCRF_TEST_DATA_FILE))
    dataset = dd['dataset']
    thetas = dd['thetas']

# -- Start classification ------------------------------------------------

for seq in range(len(dataset)):

    # -- with pycrfsuite
    s_ = tagger.tag(data['X'][seq])
    y_ = np.array([int(model.labels[s]) for s in s_])
    prob_ = tagger.probability(s_)

    print "\n-- With crfsuite:"
    print "labels:\n", s_, "\n", y_
    print "probability:\t %f" % prob_

    # -- with flexcrf
    f_xy, y = dataset[seq]
    
    theta = thetas[seq]

    m_xy, f_m_xy = _compute_all_potentials(f_xy, theta)

    y_pred = viterbi_decoder(m_xy)

    alpha = _forward_score(m_xy)
    #beta = _backward_score(m_xy)
    z_x = _partition_fun_value(alpha)

    # compare flexcrf prob to crfsuill2002-esp.crfsuite-te prob
    f_x = _feat_fun_values(f_xy, y_, with_f_x_sum=False)
    prob0 = np.exp(_posterior_score(f_x=f_x, theta=theta, z_x=z_x))
    print "flexcrf prob:\t %f" % prob0

    f_x = _feat_fun_values(f_xy, y_pred, with_f_x_sum=False)
    prob = np.exp(_posterior_score(f_x=f_x, theta=theta, z_x=z_x))

    print "-- With flexcrf:"
    print "labels:\n", y_pred
    print "equal predictions: ", all(y_pred == y_)
    print "probability:\t %f" % prob
    print "delta:\t %f" % abs(prob-prob_)

tagger.close()